In [1]:
from utils import (
    TransactionDataset,
    TuneHyperParams,
    FinalModelPerformance
)

from constants import Columns, ModelConstants, ResamplingStrategy

In [13]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    FunctionTransformer,
    Normalizer,
    PowerTransformer,
    RobustScaler
)
from sklearn.compose import make_column_transformer, make_column_selector

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import TomekLinks, NearMiss

from joblib import dump


In [3]:
data = TransactionDataset().get_training_test_split()

In [4]:
log_transform = FunctionTransformer(
    func=np.log1p, inverse_func=np.expm1, check_inverse=False
)

order_mapping = [['0', '0.2', '0.4', '0.6', '0.8', '1']]

norm_and_one_hot_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (Normalizer(), make_column_selector(dtype_include="number"))
)

log_and_ordinal_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (log_transform, make_column_selector(dtype_include="number"))
)

power_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (PowerTransformer(), make_column_selector(dtype_include="number"))
)

robust_transformation = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (RobustScaler(), make_column_selector(dtype_include="number"))
)

In [5]:
knn_classifier = KNeighborsClassifier(
    n_jobs=ModelConstants.N_JOBS
)

In [6]:
smote_sampling = SMOTE(random_state=ModelConstants.RANDOM_STATE)
adasyn_sampling = ADASYN(random_state=ModelConstants.RANDOM_STATE)

In [7]:
near_miss_sampling = NearMiss()
tomek_link_sampling = TomekLinks()

In [8]:
column_transforms_only_model = Pipeline(
    [
        ("column_transformation", log_and_ordinal_transforms),
        ("over_sampling", smote_sampling),
        ("under_sampling", near_miss_sampling),
        ("knn", knn_classifier)
    ]
)

In [9]:
column_transformation_tune = {
    "column_transformation": [
        log_and_ordinal_transforms,
        norm_and_one_hot_transforms,
        power_transforms,
        robust_transformation,
    ]
}
over_sampling_tuning = {
    "over_sampling": [
        smote_sampling,
        adasyn_sampling,
    ]
}
over_sampling__sampling_strategy_tuning = {
    "over_sampling__sampling_strategy": [
        ResamplingStrategy.MINORITY_ONLY,
        ResamplingStrategy.ALL,
        0.2,
        0.3,
        0.4,
        0.5,
        0.6,
        0.7
    ],
}
under_sampling_tuning = {
    "under_sampling": [
        near_miss_sampling,
        tomek_link_sampling,
        "passthrough"
    ],
}
knn__n_neighbors_tuning = {
    "knn__n_neighbors": [5, 7, 9, 11, 13, 15, 17, 19, 21, 25]
}
knn__weights = {"knn__weights": ["uniform", "distance"],
}

tuning_params_1 = (
        column_transformation_tune |
        over_sampling_tuning |
        over_sampling__sampling_strategy_tuning |
        under_sampling_tuning |
        knn__n_neighbors_tuning |
        knn__weights
)

tuning_params_2 = (
        column_transformation_tune |
        under_sampling_tuning |
        knn__n_neighbors_tuning |
        knn__weights
)

In [10]:
best_model = (
    TuneHyperParams()
    .random_grid_search(
     column_transforms_only_model,
     [tuning_params_1, tuning_params_2]
    )
    .fit_model(
        data.TRAINING.predictors,
        data.TRAINING.outcome
    )
    .get_best_model()
)

{   'column_transformation': ColumnTransformer(transformers=[('onehotencoder', OneHotEncoder(),
                                 ['Customer_Type']),
                                ('ordinalencoder',
                                 OrdinalEncoder(categories=[['0', '0.2', '0.4',
                                                             '0.6', '0.8',
                                                             '1']]),
                                 ['SpecificHoliday']),
                                ('robustscaler', RobustScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x13dc9b4c0>)]),
    'knn__n_neighbors': 21,
    'knn__weights': 'uniform',
    'over_sampling': SMOTE(random_state=123, sampling_strategy=0.6),
    'over_sampling__sampling_strategy': 0.6,
    'under_sampling': TomekLinks()}
Best parameter (CV score: 0.612):


In [11]:
final_model_knn = FinalModelPerformance(
    model=best_model,
    data=data
)

final_model_knn.get_cross_validation_results()

,Metric for Training Set,Score
0,fit_time mean score,0.150899
1,score_time mean score,0.113099
2,5-fold CV F1 Score mean score,0.611603
3,5-fold CV Accuracy mean score,0.851189
4,5-fold CV Balanced Accuracy mean score,0.809837
5,5-fold CV Matthew's Correlation Coefficient me...,0.537465


In [12]:
final_model_knn.get_final_model_performance()

,Metric for Testing Set,Score
0,F1 Score,0.609626
1,Accuracy,0.850471
2,Balanced Accuracy,0.731997
3,Matthew's Correlation Coefficient,0.534584


In [14]:
dump(final_model_knn.model, "./models_exports/knn_classifier.joblib")

['./models_exports/knn_classifier.joblib']